In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# Introduction


This is the for 

* module `fieldnn.basicnn.learner` module
* module `fieldnn.configfn.learnerfn` module

# Prepare Batch

In [2]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
# l

recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

data/ProcData/TensorFolder/Task2YearXXX


In [3]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
    print(tensor_folder)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

data/ProcData/TensorFolder/Task2YearXXX/P@age-AgeNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX/P@basicInfo-basicInfoDftGrn
data/ProcData/TensorFolder/Task2YearXXX/EC@BasicInfo-BasicDftGrn
data/ProcData/TensorFolder/Task2YearXXX/EC@DT_min-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/A1C@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/A1C@V-A1CNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX/Diag@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX/Diag@Value-DiagDftGrn
data/ProcData/TensorFolder/Task2YearXXX/PNSectSent@Sentence-Tk@TknzLLMGrn
B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [4]:
full_recfldgrn_list = [k for k in batch_rfg]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [5]:
import torch 

RECFLD_TO_TENSOR = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
   
    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
    
    RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    

In [10]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 



default_E_subunit_name = 'E'
df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_E_subunit_name)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
1,E,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'Male': 1...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
2,E,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'X': 1, '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
3,E,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
4,E,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
5,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
6,E,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
7,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
8,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': BertTokenizerFast(name_or...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [11]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

# Get Input

In [13]:
for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


# Learner: BasicNN Name

## TFM

### Module

In [14]:
import torch
import numpy as np
from fieldnn.utils.layerfn import orderSeq, restoreSeq

def _addindent(s_, numSpaces):
    s = s_.split('\n')
    # don't do anything for single-line stuff
    if len(s) == 1:
        return s_
    first = s.pop(0)
    s = [(numSpaces * ' ') + line for line in s]
    s = '\n'.join(s)
    s = first + '\n' + s
    return s

class TFMLayer(torch.nn.Module):
    def __init__(self, 
                 input_size = 200, 
                 output_size = 200, # d_model
                 nhead = 8,
                 num_encoder_layers = 6, # only have encoder part
                 num_decoder_layers = 0, # in default, we don't need decoder part. 
                 dim_feedforward = 2048, 
                 tfm_dropout = 0.1,
                 tfm_activation = 'relu', 
                 psn_max = 512, 
                 psn_embedprocess = {}):
        
        '''https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/transformer.py'''

        super(TFMLayer,self).__init__()
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.input_size = input_size
        self.tfm_input_size = input_size
        self.n_directions = 1
        self.output_size = output_size
        assert output_size % self.n_directions == 0 
        self.hidden_size = int(output_size / self.n_directions)
        assert self.hidden_size == self.tfm_input_size
        # self.psn_size = psn_size 
        
        
        
        self.dim_feedforward = dim_feedforward
        
        self.transformer  = torch.nn.Transformer(d_model = self.hidden_size, 
                                                 nhead = nhead,
                                                 num_encoder_layers = self.num_encoder_layers,
                                                 num_decoder_layers = self.num_decoder_layers,
                                                 dim_feedforward = dim_feedforward, 
                                                 dropout = tfm_dropout,
                                                 activation = tfm_activation,
                                                 batch_first = True,
                                                 # src_mask_flag = False, # see all tokens in a sentence 
                                                 # # This IS THE NEW PART. NOT PyTorch.nn.
                                                 )
        
        # Part a: PSN embedding
        # (bs, xxx, psn_max, dim)
        self.psn_max = psn_max
        self.psn_embedding = torch.nn.Embedding(self.psn_max + 1, 
                                                self.input_size, 
                                                padding_idx = 0)
        
        # Part b: PSN EmbedProcess
        self.psn_embedprocess = torch.nn.ModuleDict()
        for method, config in psn_embedprocess.items():
            if method == 'dropout':
                self.psn_embedprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.psn_embedprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
            else:
                raise ValueError(f'no avialable embedprocess method {method}')
                
        # you can either choose v1 or v2 forward method.
        # self.forward = self.forward_v1
        self.forward = self.forward_v2
    
    def forward_v1(self, holder, info):
        
        # (1) get the leng_mask
        leng_mask = holder == 0
        
        
        # (2.1) get the psn_embed 
        psn_id = self.generate_psnidx(leng_mask) 
        psn_embed = self.psn_embedding(psn_id)
        # print(psn_embed[0, 0, 0, :, 0], '<------------- psn_embed 1')
        
        # (2.2) TODO: process psn_embed? Do we need the further embed process? 
        for nn, layer in self.psn_embedprocess.items(): 
            psn_embed = layer(psn_embed)
            
        # print(psn_embed[0, 0, 0, :, 0], '<------------- psn_embed 2')
        
        # (2.3) add psn_embed to info
        info = info + psn_embed
        # print(info[0, 0, 0, :, 0], '<------------- info = info + psn_embed')
        
        
        # (3) reshape
        ord_info, ord_leng_mask, r_ix = self.reshape(info, leng_mask)
        
        # print(ord_info[0, :, 0], '<------------- ord_info')
        
        # (4) do the transformer calculator
        ord_info_output = self.transformer(ord_info, ord_info, 
                                           src_key_padding_mask = ord_leng_mask,  
                                           tgt_key_padding_mask = ord_leng_mask)
        
        # print(ord_info_output[0, :, 0], '<------------- ord_info_output')
        
        # (5) restore
        info = self.restore(ord_info_output, leng_mask, r_ix)
        # print(info[0, 0, 0, :, 0], '<------------- info = self.restore(ord_info_output, leng_mask, r_ix)')
            
        return holder, info
    
    
    def forward_v2(self, holder, info):
        # (1) get the leng_mask
        leng_mask = holder == 0
        
        # print(info[0, 0, 0, :, 0], '<------------- info 1')
        
        
        # (2) reshape 
        ord_info, ord_leng_mask, r_ix = self.reshape(info, leng_mask)
        # print(ord_info[0, :, 0], '<------------- ord_info')
        
        # (3.1) get the psn_embed 
        psn_id = self.generate_psnidx(ord_leng_mask) 
        psn_embed = self.psn_embedding(psn_id)
        # print(psn_embed[0, :, 0], '<------------- psn_embed 1')
        
        
        # (3.2) TODO: process psn_embed? Do we need the further embed process? 
        for nn, layer in self.psn_embedprocess.items(): 
            psn_embed = layer(psn_embed)
            
        # print(psn_embed[0, :, 0], '<------------- psn_embed 2')
        
        
        # (3.3) add psn_embed to info
        ord_info = ord_info + psn_embed
        
        # print(ord_info[0, :, 0], '<------------- ord_info 1')
        
    
        # (4) do the transformer calculator
        ord_info_output = self.transformer(ord_info, ord_info, 
                                           src_key_padding_mask = ord_leng_mask,  
                                           tgt_key_padding_mask = ord_leng_mask)
        
        # print(ord_info_output[0, :, 0], '<------------- ord_info_output 1')
        
        
        # (5) restore
        info = self.restore(ord_info_output, leng_mask, r_ix)
        # print(info[0, 0, 0, :, 0], '<------------- info 2')
        
        return holder, info
    
    
    
    def reshape(self, info, leng_mask):
        nbs = int(np.array(info.shape[:-2]).prod())
        # nbs = np.array(info.shape[:-2]).prod()
        ngrn, dim = info.shape[-2:]
        # print(nbs, ngrn, dim)
        
        tmp_info = info.contiguous().view(nbs, ngrn, dim)
        # print(tmp_info.shape)

        tmp_leng_mask = leng_mask.contiguous().view(nbs, ngrn)
        # print(tmp_leng_mask.shape)

        tmp_leng = (tmp_leng_mask == 0).sum(-1)
        # print(tmp_leng.shape)
        
        ord_info,      ord_leng, r_idx = orderSeq(tmp_info, tmp_leng)
        ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
        return ord_info, ord_leng_mask, r_idx
    
    def restore(self, ord_info_output, leng_mask, r_idx):
        info_new = restoreSeq(ord_info_output, r_idx)
        output_size = info_new.shape[-1]
        info_output = info_new.view(*list(leng_mask.shape) + [output_size])
        return info_output
        
    def generate_psnidx(self, leng_mask):
        # leng_mask = holder == 0
        psn_idx = (leng_mask == False).cumsum(-1).masked_fill(leng_mask, 0)
        return psn_idx
    
    
    def __repr__(self):
        # We treat the extra repr like the sub-module, one item per line
        extra_lines = []
        extra_repr = self.extra_repr()
        # empty string will be split into list ['']
        if extra_repr:
            extra_lines = extra_repr.split('\n')
        child_lines = []
        for key, module in self._modules.items():
            mod_str = repr(module)
            mod_str = _addindent(mod_str, 2)
            child_lines.append('(' + key + '): ' + mod_str)
        lines = extra_lines + child_lines

        main_str = self._get_name() + f'LEARNER(TFM): input({self.input_size}), output({self.output_size}): ('
        lines = [f'(Encoder): EncoderLayer(layers_num={self.num_encoder_layers}, dim_feedforward={self.dim_feedforward})', 
                 f'(Decoder): DecoderLayer(layers_num={self.num_decoder_layers}, dim_feedforward={self.dim_feedforward})']
        main_str += '\n  ' + '\n  '.join(lines) + '\n' + ')'
        
        # if lines:
        #     # simple one-liner info, which most builtin Modules will use
        #     if len(extra_lines) == 1 and not child_lines:
        #         main_str += extra_lines[0]
        #     else:
        #         main_str += '\n  ' + '\n  '.join(lines) + '\n'
        # main_str += ')'
        return main_str
    

### Config

In [15]:
def get_tfm_para(input_size, default_tfm_para):
    assert 'input_size' not in default_tfm_para
    
    tfm_para =  {'input_size': input_size,
                 'output_size': input_size,
                 'nhead': 8,
                 'num_encoder_layers': 6,
                 'num_decoder_layers': 0,
                 'dim_feedforward': 2048,
                 'tfm_dropout': 0.1,
                 'tfm_activation': 'relu',
                 'psn_max': 512, 
                 'psn_embedprocess': {}
                }
    
    for k, v in default_tfm_para.items(): tfm_para[k] = v
    return tfm_para

### Usage

**recfld information**

In [16]:
for recfld, EmbedTensor in RECFLD_TO_EMBEDTESNOR.items():
    print(recfld, EmbedTensor['info'].shape)

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


**get configuration**

In [17]:
### Main Config
### others will change with each recfldgrn

#######################
psn_embedprocess = {
    # 'activator': 'gelu',
    # 'dropout': {'p': 0.5, 'inplace': False}, # maybe the dropout?
    # 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}
}

embed_size = 128
default_tfm_para = {'psn_max': 512, 
                    'psn_embedprocess': psn_embedprocess}
#######################
tfm_para = get_tfm_para(embed_size, default_tfm_para)
tfm_para

{'input_size': 128,
 'output_size': 128,
 'nhead': 8,
 'num_encoder_layers': 6,
 'num_decoder_layers': 0,
 'dim_feedforward': 2048,
 'tfm_dropout': 0.1,
 'tfm_activation': 'relu',
 'psn_max': 512,
 'psn_embedprocess': {}}

**init model**

In [18]:
NN = TFMLayer(**tfm_para)
# tfm_layer
NN

TFMLayerLEARNER(TFM): input(128), output(128): (
  (Encoder): EncoderLayer(layers_num=6, dim_feedforward=2048)
  (Decoder): DecoderLayer(layers_num=0, dim_feedforward=2048)
)

**prepare input**

In [19]:
# recfld = 'B-P-EC-Diag@Value-DiagDft'

recfld = 'B-P@age-AgeNumeDft'
info_dict = RECFLD_TO_EMBEDTESNOR[recfld]
holder, info = info_dict['holder'], info_dict['info']

print(holder.shape, '<-----  holder.shape')
print(info.shape, '<-----  info.shape')

torch.Size([4, 19]) <-----  holder.shape
torch.Size([4, 19, 128]) <-----  info.shape


**I-NN-O**

In [20]:
holder_output, info_output_v1 = NN.forward_v1(holder, info)
# info_output_v1
if len(info.shape) == 5:
    print(info_output_v1[0, 0, 0, :, 0])
if len(info.shape) == 3:
    print(info_output_v1[0, :, 0])

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0469,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], grad_fn=<SelectBackward0>)


In [21]:
holder_output, info_output_v2 = NN.forward_v2(holder, info)
if len(info.shape) == 5:
    print(info_output_v2[0, 0, 0, :, 0])
if len(info.shape) == 3:
    print(info_output_v2[0, :, 0])

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0469,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], grad_fn=<SelectBackward0>)


In [22]:
holder_output, info_output = NN(holder, info)
if len(info.shape) == 5:
    print(info_output[0, 0, 0, :, 0])
if len(info.shape) == 3:
    print(info_output[0, :, 0])

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0469,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], grad_fn=<SelectBackward0>)


## Linear

### Module

In [19]:
import torch

class LinearLayer(torch.nn.Module):

    def __init__(self, 
                 input_size  = 200, 
                 output_size = 200,
                 initrange = 0.1
                 ):

        super(LinearLayer, self).__init__()
    
        self.input_size  = input_size
        self.output_size = output_size
        self.linear  = torch.nn.Linear(self.input_size, self.output_size)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    
    def forward(self, holder, info):
        # assert self.input_fullname == fullname
        # (1) learn the info
        info = self.linear(info)
        
        # (2) do the masked_leng because of non-zero bias
        leng_mask = holder == 0
        info = info.masked_fill(leng_mask.unsqueeze(-1), 0)
    
        # (3) post-process
        # for nn_name, layer in self.postprocess.items():
        #     info = layer(info)
            
        # we do not change the fullname and holder
        return holder, info

### Config

In [20]:
default_linear_para = {'initrange': 0.1}

def get_linear_para(input_size, output_size, default_linear_para):
    linear_para =  {'input_size': input_size,
                    'output_size': output_size}
    
    for k, v in default_linear_para.items(): linear_para[k] = v
    return linear_para

### Usage

**recfld information**

In [21]:
for recfld, EmbedTensor in RECFLD_TO_EMBEDTESNOR.items():
    print(recfld, EmbedTensor['info'].shape)

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


**get configuration**

In [22]:
############
embed_size = embed_size
default_linear_para = {'initrange': 0.1}
############

linear_para = get_linear_para(embed_size, embed_size, default_linear_para)
linear_para

{'input_size': 128, 'output_size': 128, 'initrange': 0.1}

**init model**

In [23]:
NN = LinearLayer(**linear_para)
NN

LinearLayer(
  (linear): Linear(in_features=128, out_features=128, bias=True)
)

**prepare input**

In [24]:
recfld = 'B-P@basicInfo-basicInfoDft'
info_dict = RECFLD_TO_EMBEDTESNOR[recfld]
info, holder = info_dict['info'], info_dict['holder']
print(holder.shape, '<-----  holder.shape')
print(info.shape, '<-----  info.shape')


torch.Size([4, 2]) <-----  holder.shape
torch.Size([4, 2, 128]) <-----  info.shape


**I-NN-O**

In [25]:
# Problem: every time I run it, the result will change. 
holder_output, info_output = NN(holder, info)
print(info_output.shape)
# info_output.shape
info_output

torch.Size([4, 2, 128])


tensor([[[-1.0211, -0.9180, -0.6459,  ...,  0.5810, -0.5451, -0.2727],
         [ 0.9918, -1.0760, -0.8198,  ...,  1.0739, -0.2542, -0.6560]],

        [[-1.1719,  0.8070, -0.3654,  ..., -0.3696, -0.0316,  0.0692],
         [ 0.3357, -0.1931, -0.4071,  ...,  0.3242, -0.1918, -0.7566]],

        [[-0.1401,  0.2694,  1.1852,  ..., -0.2132, -0.8237,  0.1233],
         [ 0.4198,  0.5447, -0.8063,  ...,  0.2503,  0.1053, -0.5188]],

        [[ 0.0639,  0.5080,  0.8615,  ..., -0.4055, -0.2118, -0.0889],
         [-2.0946, -1.2461, -0.3083,  ...,  0.5666,  1.5504,  0.9192]]],
       grad_fn=<MaskedFillBackward0>)

# Learner: Basic NN Type

## Module

In [26]:
import os
import torch
import numpy as np

# from fieldnn.basicnn.learner.tfm import TFMLayer
# from fieldnn.basicnn.learner.linear import LinearLayer

# from fieldnn.utils.layerfn import orderSeq, restoreSeq, align_psn_idx, get_Layer2Holder
# from fieldnn.utils.parafn import generate_psn_embed_para


class Learner_Layer(torch.nn.Module):
    def __init__(self, 
                 input_names_nnlvl, output_name_nnlvl, learner_layer_para,
                ):
        super(Learner_Layer, self).__init__()
        
        # Part 0: Meta
        # here input_names and out_tensor just the tensor name, 
        # intead, the info_dict contains the corresponding real tensors.
        assert len(input_names_nnlvl) == 1
        self.input_names_nnlvl = input_names_nnlvl
        self.input_name_nnlvl = input_names_nnlvl[0]
        
        # output_name should be generated from the input_names
        self.output_name_nnlvl = output_name_nnlvl
        
        # the input feature dim size and output feature dim size
        self.input_size = learner_layer_para['input_size']
        self.output_size = learner_layer_para['output_size']
 
        # Part 1: NN
        nn_name, nn_para = learner_layer_para['nn_name'], learner_layer_para['nn_para']
        
        if nn_name.lower() == 'tfm':
            assert self.input_size == self.output_size
            self.Learner = TFMLayer(**nn_para)
        elif nn_name.lower() == 'linear':
            self.Learner = LinearLayer(**nn_para)
        # elif nn_name.lower() == 'cnn':
        #     self.Learner = CNNLayer(**nn_para)
        # elif nn_name.lower() == 'rnn':
        #     self.Learner = RNNLayer(**nn_para)
        else:
            raise ValueError(f'NN "{nn_name}" is not available')
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in learner_layer_para['postprocess'].items():
            if method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
        # self.Ignore_PSN_Layers = learner_layer_para['Ignore_PSN_Layers']
        
    # def get_psn_embed_tensor(self, fullname, holder):
    #     name = fullname.split('-')[-1]
    #     Layer2Idx = {v:idx for idx, v in enumerate(fullname.split('-'))}
    #     Layer2Holder = get_Layer2Holder(fullname, holder, self.Ignore_PSN_Layers)
    #     psn_embed = 0
    #     for source_layer, Embed in self.PSN_Embed_ModuleDict.items():
    #         cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
    #         psn_embed = psn_embed + Embed(cpsn_idx)
    #     return psn_embed
    
    def forward(self, input_names_nnlvl, INPUTS_TO_INFODICT):
        # information preparation.
        # 'INPUTS_TO_INFODICT` will come from SubUnit Layer.
        assert len(input_names_nnlvl) == 1
        input_name_nnlvl = input_names_nnlvl[0]
        assert self.input_name_nnlvl == input_name_nnlvl
        
        info_dict = INPUTS_TO_INFODICT[input_name_nnlvl]
        holder, info = info_dict['holder'], info_dict['info']
        
        # print(holder.shape, info.shape)
        # the following part is the data proprocessing
        holder, info = self.Learner(holder, info)
        
        for name, layer in self.postprocess.items():
            info = layer(info)
            
        # we don't need to change the holder here.
        return self.output_name_nnlvl, {'holder': holder, 'info': info}
    

## Config

In [27]:
def get_learner_para(nn_name, default_nn_para, 
                     input_size, output_size, 
                     postprocess):
    
    learner_para = {}
    
    # (1) 
    learner_para['nn_type'] = 'learner'  
    learner_para['nn_name'] = nn_name # TFM or linear
    
    # (2)
    if nn_name.lower() == 'tfm':
        para = get_tfm_para(input_size, default_nn_para)
    elif nn_name.lower() == 'linear':
        para = get_linear_para(input_size, output_size, default_nn_para)
    else:
        raise ValueError(f'The NN "{nn_name}" is not available yet')
    learner_para['nn_para'] = para
    
    # (3)
    learner_para['input_size'] = input_size
    learner_para['output_size'] = output_size
    

    # (4) Post Process
    learner_para['postprocess'] = postprocess
    
    return learner_para

## Usage

In [28]:


#######################
psn_embedprocess = {# 'activator': 'gelu',
                    'dropout': {'p': 0.5, 'inplace': False},
                    'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}

process = {'activator': 'gelu',
            'dropout': {'p': 0.5, 'inplace': False},
            'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}


embed_size = 128
default_tfm_para = {'psn_max': 512, 
                    'psn_embedprocess': psn_embedprocess}
#######################



nn_name = 'tfm'
default_nn_para = default_tfm_para
input_size = embed_size
output_size = embed_size
postprocess = process


learner_para = get_learner_para(nn_name, default_nn_para, 
                                 input_size, output_size, 
                                 postprocess)

learner_para

{'nn_type': 'learner',
 'nn_name': 'tfm',
 'nn_para': {'input_size': 128,
  'output_size': 128,
  'nhead': 8,
  'num_encoder_layers': 6,
  'num_decoder_layers': 0,
  'dim_feedforward': 2048,
  'tfm_dropout': 0.1,
  'tfm_activation': 'relu',
  'psn_max': 512,
  'psn_embedprocess': {'dropout': {'p': 0.5, 'inplace': False},
   'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}},
 'input_size': 128,
 'output_size': 128,
 'postprocess': {'activator': 'gelu',
  'dropout': {'p': 0.5, 'inplace': False},
  'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}

In [29]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [30]:
recfld = 'B-P-EC-Diag@Value-DiagDft'
recfld = 'B-P@age-AgeNumeDft'

In [31]:
input_names_nnlvl = [recfld]
output_name_nnlvl = recfld

NN = Learner_Layer(input_names_nnlvl, output_name_nnlvl, learner_para)
NN

Learner_Layer(
  (Learner): TFMLayerLEARNER(TFM): input(128), output(128): (
    (Encoder): EncoderLayer(layers_num=6, dim_feedforward=2048)
    (Decoder): DecoderLayer(layers_num=0, dim_feedforward=2048)
  )
  (postprocess): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)

**recfld informaiton**

In [32]:
input_names_nnlvl

['B-P@age-AgeNumeDft']

In [33]:
INPUTS_TO_INFODICT = {}

for inp in input_names_nnlvl:
    INPUTS_TO_INFODICT[inp] = RECFLD_TO_EMBEDTESNOR[inp]
    

In [34]:

# information preparation.
# 'INPUTS_TO_INFODICT` will come from SubUnit Layer.
assert len(input_names_nnlvl) == 1
input_name_nnlvl = input_names_nnlvl[0]
assert NN.input_name_nnlvl == input_name_nnlvl

In [35]:
info_dict = INPUTS_TO_INFODICT[input_name_nnlvl]
holder, info = info_dict['holder'], info_dict['info']
print(info.shape)

torch.Size([4, 19, 128])


In [36]:
# print(holder.shape, info.shape)
# the following part is the data proprocessing
holder, info = NN.Learner(holder, info)

for name, layer in NN.postprocess.items():
    info = layer(info)

In [37]:
outputname, info_dict = NN(input_names_nnlvl, INPUTS_TO_INFODICT)

info_dict['info'].shape

torch.Size([4, 19, 128])